# Notebook 4: One-Class SVM para Detecção de Novidades

## Objetivo
Diferenciar **Anomalia** de **Novidade** e usar One-Class SVM (OCSVM).

## O Conceito
- **Detecção de Outliers (Não supervisionada)**: Temos dados sujos (normais + anômalos) e queremos limpar.
- **Detecção de Novidades (Semi-supervisionada)**: Temos dados LIMPOS (somente o que é normal) no treino. Queremos detectar se um *novo* dado no futuro discrepa desse padrão conhecido.

O OCSVM aprende a fronteira (boundary) do que é "Normal". Qualquer coisa fora dessa fronteira é classificada como anomalia.

In [ ]:
!pip install -q pyod pandas matplotlib seaborn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyod.models.ocsvm import OCSVM
from pyod.utils.data import generate_data

plt.rcParams['figure.figsize'] = (10, 6)

## 1. Cenário: Logs de Acesso
Vamos supor que temos históricos de logins bem-sucedidos e normais (treino). 
- Feature 1: Horário do login (normalizado)
- Feature 2: Duração da sessão

No treino, só temos comportamentos normais. No teste, introduzimos invasores.

In [ ]:
# Gerando dados DE TREINO sem contaminação (apenas dados bons)
X_train, _ = generate_data(n_train=400, n_test=0, n_features=2, contamination=0, random_state=42)

# Gerando dados DE TESTE com ataques
# Usamos contamination=0.2 para simular que 20% do teste são ataques
_, X_test, _, y_test = generate_data(n_train=0, n_test=200, n_features=2, contamination=0.2, random_state=42)

print(f"Treino (Limpo): {X_train.shape}")
print(f"Teste (Sujo): {X_test.shape}")

## 2. Treinando OCSVM
O OCSVM tenta encontrar a menor hiper-esfera (ou hiper-plano no kernel RBF) que englobe os dados de treino.

In [ ]:
# nu: parametro chave. Representa o limite superior da fração de erros de treinamento 
# (ou fração de pontos fora da fronteira). nu=0.05 significa que permitimos até 5% de 'falsos normais' ou que esperamos ser muito rigidos.
clf_ocsvm = OCSVM(kernel='rbf', nu=0.05)

clf_ocsvm.fit(X_train)

## 3. Prevendo Novidades (Ataques)
Agora aplicamos o modelo treinado nos dados de teste.

In [ ]:
y_test_pred = clf_ocsvm.predict(X_test)
y_test_scores = clf_ocsvm.decision_function(X_test)

from pyod.utils.data import evaluate_print
evaluate_print('OCSVM', y_test, y_test_scores)

## 4. Visualizando a Fronteira
Vamos visualizar onde o modelo desenhou a linha de corte.

In [ ]:
# Criando um grid para desenhar a fronteira de decisão
xx, yy = np.meshgrid(np.linspace(-4, 4, 100), np.linspace(-4, 4, 100))
Z = clf_ocsvm.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.figure(figsize=(10, 6))

# Contorno da decisão (Fronteira)
plt.contour(xx, yy, Z, levels=[0], linewidths=2, colors='black')

# Dados Normais (Treino)
plt.scatter(X_train[:, 0], X_train[:, 1], c='blue', s=20, label='Treino (Limpo)')

# Dados de Teste (Anomalias detectadas em vermelho)
mask_anomalia = y_test == 1
plt.scatter(X_test[mask_anomalia, 0], X_test[mask_anomalia, 1], c='red', marker='x', s=50, label='Ataques Reais (Teste)')

plt.legend()
plt.title('Fronteira de Decisão OCSVM - Dados de Login')
plt.show()

## Conclusão
OCSVM é ideal quando você tem certeza de que seu dataset de treino é confiável e livre de fraudes (ou quase livre), e você quer blindar o sistema contra qualquer coisa que desvie desse padrão validado.